In [1]:
# Cell 1: Agent-Specialized Dataset Curation Infrastructure
# Implements Idea 1 (Specialized Curation) through model execution and response storage
# ------------------------------------------------------------
import subprocess
import time
import pickle
import os

def run_local_ollama_model(model_name, prompt, timeout=60):
    """Core function for agent-specific dataset generation"""
    start_time = time.time()
    try:
        process = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            capture_output=True,
            text=True,
            timeout=timeout
        )
    except subprocess.TimeoutExpired:
        return "Error: Timeout - agent specialization may require adjusted parameters"
    elapsed_time = time.time() - start_time
    print(f"Execution time: {elapsed_time:.2f} seconds")
    return process.stdout.strip()

def append_response_to_pickle(new_response, filename="responses.pkl"):
    """Persistent storage for iterative dataset refinement"""
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            responses = pickle.load(f)
    else:
        responses = []    
    responses.append(new_response)    
    with open(filename, "wb") as f:
        pickle.dump(responses, f)
    print(f"Agent response archived to {filename}.")



In [2]:
# Cell 2: Modular RAG Pipeline Implementation
# Implements Idea 2 (Modular Architecture) through prompt engineering
# ------------------------------------------------------------
model = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L"

def agent_interaction_loop():
    """Modular prompt construction for different agent specializations"""
    print("Multi-agent prompt interface active. Type 'exit' to return to control.")
    while True:
        user_prompt = input("Agent input: ")
        if user_prompt.lower() in ["exit", "quit"]:
            print("Returning to system level...")
            break
        
        # Chain-of-thought prompt engineering
       # Modified prompt template in Cell 2
        augmented_prompt = (
            "Before answering, consider these related insights:\n"
            f"{insight_engine.find_similar_insights(user_prompt)}\n\n"
            "Now analyze as [Agent Role]..."
)
        
        response = run_local_ollama_model(model, augmented_prompt)
        print(f"Specialized Agent Response:\n{response}")
        append_response_to_pickle(response)

In [4]:
# Cell 3: Iterative Validation System
# Implements Idea 3 (Validation & Refinement) through response analysis
# ------------------------------------------------------------
def validate_responses(filename="responses.pkl"):
    """Continuous evaluation framework for multi-agent outputs"""
    if not os.path.exists(filename):
        print("No responses to validate - run agent interactions first")
        return
    
    with open(filename, "rb") as f:
        agent_responses = pickle.load(f)
    
    print("\n--- Validation Report ---")
    print(f"Analyzing {len(agent_responses)} agent responses:")
    
    # Simple validation metrics (expand with domain-specific checks)
    avg_length = sum(len(r) for r in agent_responses)/len(agent_responses)
    unique_tokens = len(set(" ".join(agent_responses).split()))
    
    print(f"- Average response length: {avg_length:.1f} chars")
    print(f"- Unique vocabulary: {unique_tokens} tokens")
    print("- Consistency flags:", ["⚠️" if len(r)<100 else "✅" for r in agent_responses[-5:]])
    
    # Prepare for dataset refinement
    print("\nRecommendations:")
    print("- Run adversarial_cross_testing() for gap analysis")
    print("- Use prune_responses() to remove low-quality entries")

# Start the multi-agent system
if __name__ == "__main__":
    agent_interaction_loop()
    validate_responses()


Multi-agent prompt interface active. Type 'exit' to return to control.
Execution time: 21.81 seconds
Specialized Agent Response:
Response: *I explain the reasoning process in detail.*

Reasoning Process: To analyze and provide an answer to the query, I first need to understand what information is given and what it means. This allows me to determine my initial approach. Then, I break down the problem into smaller sub-problems that can be solved independently. For each sub-problem, I consider different angles, looking for patterns or insights. 

I then synthesize these solutions together to reach a final conclusion. Along the way, I make assumptions and test them against known facts to ensure they are accurate. Whenever an assumption isn't supported by evidence, I modify my approach accordingly until progress is made. Throughout this process, if new information emerges that changes things, I incorporate it into the analysis without losing sight of the original goals or constraints.

*Nex

KeyboardInterrupt: 

In [4]:
# Cell 4: Embedding Generation & Insight Sharing System
# Creates embeddings from stored responses and enables agent access
# ------------------------------------------------------------
from sentence_transformers import SentenceTransformer
import numpy as np

class AgentInsightEngine:
    def __init__(self, pickle_file="responses.pkl"):
        self.responses = self._load_responses(pickle_file)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = self._generate_embeddings()
        
    def _load_responses(self, filename):
        """Secure access to agent response history"""
        if not os.path.exists(filename):
            return []
        with open(filename, "rb") as f:
            return pickle.load(f)
    
    def _generate_embeddings(self):
        """Convert responses to semantic embeddings"""
        return self.embedding_model.encode(self.responses)
    
    def find_similar_insights(self, query, top_k=3):
        """Enable agents to retrieve relevant past insights"""
        query_embedding = self.embedding_model.encode([query])
        similarities = np.dot(self.embeddings, query_embedding.T).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]
        return [(self.responses[i], similarities[i]) for i in top_indices]

# Cell 4a: Implementation Example
if __name__ == "__main__":
    # Initialize insight engine with agent responses
    insight_engine = AgentInsightEngine()
    
    # Example agent accessing shared knowledge
    test_query = "What's the best way maximize the beauty?"
    similar_insights = insight_engine.find_similar_insights(test_query)
    
    print(f"\nInsights for '{test_query}':")
    for idx, (response, similarity) in enumerate(similar_insights):
        print(f"\nInsight {idx+1} (Similarity: {similarity:.2f}):")
        print(response[:200] + "...")


Insights for 'What's the best way maximize the beauty?':

Insight 1 (Similarity: 0.62):
To maximize beauty in every situation through channeling my inner beauty, I would first deeply analyze what makes this moment or experience beautiful. Is it the majestic sight of a sunrise painting th...

Insight 2 (Similarity: 0.49):
To analyze "inner beauty", one must consider the depth and richness of qualities that lie within an individual, beyond mere physical appearance. Inner beauty encompasses a person's true essence, chara...

Insight 3 (Similarity: 0.35):
I would analyze the query by focusing on key concepts from Gilles Deleuze's work on cinema and image philosophy. Some key terms to consider are "cinématisme" - the idea that cinema is a specific mode ...
